In [1]:
import pandas as pd
import numpy as np

import spacy
#reformat to the proper matrix dimension (num. samples, vector dimension of 300)
from scipy.sparse import csr_matrix
from scipy.sparse import hstack

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv("Data/train_processed_tf_idf_bow.csv")
test = pd.read_csv("Data/test_processed_tf_idf_bow.csv")
sub_sample = pd.read_csv("Data/sample_submission.csv")

print (train.shape, test.shape, sub_sample.shape)



(7613, 1842) (3263, 1841) (3263, 2)


In [3]:

text_data=train['text_clean'].apply(lambda x: str(x))
text_label=train.target

In [4]:
response=text_label

In [5]:
nlp = spacy.load('en')

In [6]:
#gets the average wordvec
vec1 = [doc.vector for doc in nlp.pipe(text_data, n_threads=50)]
train['word_vec'] = list(np.array(vec1))
train.head()

,Unnamed: 0,id,keyword,location,text,target,text_clean,hashtags,mentions,links,...,ûª,ûªs,ûªt,ûªve,ûï,ûïwhen,ûïwhen saw,ûò,ûó,word_vec
0,0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...,earthquake,no,no,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.82296896, 0.22125112, 0.30233097, -0.988179..."
1,1,4,None,None,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,no,no,no,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.7361565, 0.56616133, -0.43273652, -0.078039..."
2,2,5,None,None,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...,no,no,no,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.02234808, 0.08437336, 0.35420308, -0.326989..."
3,3,6,None,None,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or...",wildfires,no,no,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.70433843, 2.0910797, 0.1534155, -1.0105444,..."
4,4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...,Alaska wildfires,no,no,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.49397433, 0.540281, -0.24060287, -1.0794047..."


In [7]:
X_train=csr_matrix(list(train['word_vec']))
X_train.shape

(7613, 96)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, response, test_size=0.5, random_state=100, stratify=response)

clf= MLPClassifier(alpha=1e-4, hidden_layer_sizes=(6,4), random_state=12,validation_fraction=0, max_iter=500)

clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(6, 4), max_iter=500, random_state=12,
              validation_fraction=0)

In [10]:
val_pred=clf.predict(X_val.toarray())

cal_error=accuracy_score(y_val,val_pred)
print(cal_error)

0.6861045442605727


In [11]:
submit = sub_sample.copy()
submit.target = y_val
submit.to_csv('submit_glove.csv',index=False)

ValueError: Length of values (3807) does not match length of index (3263)